LSTM in Pytorch

In [5]:
#library imports
!pip install torch
!pip install spacy
!pip install jovian
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6501 sha256=5dc0e8310cbd5c6c668739a16bb26b790f0ed0f581993e97031ab78755f075e6
  Stored in directory: c:\users\simmu\appdata\local\pip\cache\wheels\1b\6c\cb\f9aae2bc97333c3d6e060826c1ee9e44e46306a178e5783505
Successfully built uuid


<IPython.core.display.Javascript object>

In [6]:
#input (Basic LSTM in PyTorch with Random numbers)
x = torch.tensor([[1,2, 12,34, 56,78, 90,80],
                 [12,45, 99,67, 6,23, 77,82],
                 [3,24, 6,99, 12,56, 21,22]])

using two different models

In [7]:
model1 = nn.Embedding(100, 7, padding_idx=0)
model2 = nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True)

In [8]:
out1 = model1(x)
out2 = model2(out1)

In [9]:
print(out1.shape)
print(out1)

torch.Size([3, 8, 7])
tensor([[[-0.1138, -1.3844, -0.4573, -0.4441, -1.1819,  0.2450,  0.1639],
         [ 0.5921, -1.3083,  2.2689, -2.0425,  0.4900, -0.4369, -1.4076],
         [-0.7947,  1.3762,  0.9369, -2.0566, -1.0512, -0.3015, -0.2779],
         [ 0.1303,  1.7352, -0.3041, -0.1496, -2.0450,  0.9286,  0.1745],
         [-0.8471, -0.3860, -1.1694, -0.7194, -0.6510, -0.0369, -1.1985],
         [ 1.5536, -0.7190,  0.0508,  1.2089,  0.4229,  0.2796, -0.2056],
         [-1.5812,  0.6476, -1.2037,  0.5257,  1.3810, -0.0458,  0.6029],
         [-2.6046,  0.1465, -0.4745,  0.8798,  0.1657,  0.5388,  0.1244]],

        [[-0.7947,  1.3762,  0.9369, -2.0566, -1.0512, -0.3015, -0.2779],
         [ 0.0282,  0.6131,  0.6507, -1.5690, -0.6792, -0.1663,  0.9858],
         [ 0.5950, -1.4406, -1.5375, -0.5230,  1.2359,  0.8157, -1.5023],
         [-0.4815,  1.3777,  2.3236, -0.2003, -0.1235,  0.7554,  0.9372],
         [-0.9159,  0.1368,  0.1068,  0.1013,  0.8645, -0.8717, -0.5511],
         [-0.6

In [10]:
out, (ht, ct) = model2(out1)
print(ht)

tensor([[[-0.1589, -0.0241, -0.0439],
         [ 0.1044, -0.1058,  0.0247],
         [-0.2264,  0.2256,  0.2958]]], grad_fn=<StackBackward>)


using nn.sequential

In [11]:
model3 = nn.Sequential(nn.Embedding(100, 7, padding_idx=0),
                        nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True))

In [12]:
out, (ht, ct) = model3(x)
print(out)

tensor([[[ 0.0582,  0.0995,  0.0592],
         [ 0.2026,  0.3528, -0.3194],
         [ 0.2184,  0.2725, -0.5097],
         [ 0.3748, -0.1079, -0.3855],
         [ 0.3830, -0.0189, -0.3125],
         [ 0.2868, -0.1447, -0.2994],
         [ 0.1497,  0.0120, -0.3394],
         [ 0.2409,  0.0047, -0.0993]],

        [[ 0.1437,  0.0394, -0.2014],
         [ 0.1259,  0.1149,  0.0271],
         [ 0.1465,  0.2317,  0.0763],
         [ 0.2318, -0.0348, -0.0095],
         [ 0.3190,  0.0747, -0.3237],
         [ 0.2444, -0.1477, -0.2047],
         [ 0.1595, -0.0876, -0.0358],
         [ 0.0333, -0.0167, -0.0888]],

        [[ 0.1608,  0.1862, -0.1811],
         [ 0.0418, -0.0486,  0.0225],
         [ 0.2414,  0.0597, -0.2811],
         [ 0.1673,  0.1486,  0.0118],
         [ 0.2229,  0.0901, -0.2066],
         [ 0.3826,  0.0506, -0.2140],
         [ 0.2935, -0.3682, -0.0411],
         [ 0.1645, -0.1411, -0.0221]]], grad_fn=<TransposeBackward0>)


Multiclass Text Classification

We are going to predict item ratings based on customer reviews based on a e commerce store of clothing dataset from Kaggle

In [14]:
#loading the data
reviews = pd.read_csv("C:/Users/simmu/Downloads/archive (1)/Womens Clothing E-Commerce Reviews.csv")
print(reviews.shape)
reviews.head()

(23486, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [15]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

In [16]:
#keeping only relevant columns and calculating sentence lengths
reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews['review_length'] = reviews['review'].apply(lambda x: len(x.split()))
reviews.head()

,review,rating,review_length
0,Absolutely wonderful - silky and sexy and com...,4,8
1,Love this dress! it's sooo pretty. i happen...,5,62
2,Some major design flaws I had such high hopes ...,3,102
3,"My favorite buy! I love, love, love this jumps...",5,25
4,Flattering shirt This shirt is very flattering...,5,38


In [17]:
#changing ratings to 0-numbering
zero_numbering = {1:0, 2:1, 3:2, 4:3, 5:4}
reviews['rating'] = reviews['rating'].apply(lambda x: zero_numbering[x])

In [18]:
#mean sentence length
np.mean(reviews['review_length'])

60.832921740611425

In [29]:
#tokenization
from spacy.cli.download import download
download(model="en_core_web_sm")
tok = spacy.load('en_core_web_sm')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [30]:
#count number of occurences of each word
counts = Counter()
for index, row in reviews.iterrows():
    counts.update(tokenize(row['review']))

In [31]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 14137
num_words after: 8262


In [32]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [33]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [34]:
reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
reviews.head()

,review,rating,review_length,encoded
0,Absolutely wonderful - silky and sexy and com...,3,8,"[[2, 3, 4, 5, 6, 7, 8, 7, 9, 0, 0, 0, 0, 0, 0,..."
1,Love this dress! it's sooo pretty. i happen...,4,62,"[[2, 10, 11, 12, 5, 13, 14, 15, 16, 5, 17, 18,..."
2,Some major design flaws I had such high hopes ...,2,102,"[[54, 55, 56, 57, 17, 58, 59, 60, 61, 62, 11, ..."
3,"My favorite buy! I love, love, love this jumps...",4,25,"[[68, 109, 110, 2, 17, 10, 2, 10, 2, 10, 11, 1..."
4,Flattering shirt This shirt is very flattering...,4,38,"[[122, 123, 11, 123, 52, 92, 122, 19, 124, 125..."


In [35]:
#check how balanced the dataset is
Counter(reviews['rating'])

Counter({3: 5077, 4: 13131, 2: 2871, 1: 1565, 0: 842})

In [36]:
X = list(reviews['encoded'])
y = list(reviews['rating'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [37]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [38]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [39]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [40]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

LSTM with fixed length input

In [41]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [42]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)